In [ ]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import spacy
from urllib.parse import urljoin, urlparse
import time
from Uniscraper.Uniscraper import uniscraper


def generate_url_list(school_info, max_links=20):
    """
    This function returns the subdomain links visible from a food bank or wellness programs homepage.
    Params:
        school_info: DataFrame with 'school_name' and 'url' columns
        max_links: max size of the list being returned for each school
    Returns:
        result_df: DataFrame with 'school_name' and 'url' columns
    """
    all_links = []  

    
    for index, row in school_info.iterrows():
        #get school name and url to base/starting page
        school_name = row["school_name"] 
        url = row["url"]  

        
        driver = webdriver.Chrome() 
        driver.get(url)
        time.sleep(1)

        # parsing url to ensure consistency and proper formatting
        parsed_url = urlparse(url)
        #takes elements such as scheme and netloc to create valid base domain
        base_domain = f"{parsed_url.scheme}://{parsed_url.netloc}"

        # set data structure used to avoid duplicates
        links = set()

        #looping though each sublink
        for a in driver.find_elements(By.TAG_NAME, "a"):
            href = a.get_attribute("href")
            if href: # if link exists
                # joining to ensure only focused websites are being generated
                full_link = urljoin(base_domain, href)
                #adding to list of links if it has base domain 
                if full_link.startswith(base_domain) and full_link not in links:
                    links.add(full_link)
                    if len(links) >= max_links: #stopping point after max_links
                        break

        
        driver.quit()

        # all links for a school will have school name but different urls
        for link in links:
            all_links.append({"school_name": school_name, "url": link})

    # conver to dataframe 
    result_df = pd.DataFrame(all_links)

    return result_df



# small example of dataframe for 5 schools
school_info = pd.DataFrame({
    "school_name": ["UNC", "UGA","UC Davis", "UCLA", "PSU"], 
    "url": ["https://dos.unc.edu/student-support/basicneeds/",
            "https://well-being.uga.edu/basic-needs/#:~:text=YOUR%20BASIC%20WELL-BEING%20NEEDS&text=Access%20to%20essential%20resources%20is,to%20students%20at%20no%20cost",
            "https://financialaid.ucdavis.edu/wellness-outreach/basic-needs",
            "https://bewellbruin.ucla.edu/resource/ucla-basic-needs",
            "https://studentaffairs.psu.edu/basic-needs-support"
            ]  
})

result = generate_url_list(school_info)

# Load NLP model
nlp = spacy.load("en_core_web_sm")

#Urls and school names list 
university_urls = result["url"]
school_names = result["school_name"]


keywords = [
    "Food Security", "Housing Stability", "Financial Assistance", "Healthcare Services", "Mental Health Support",
    "Transportation Access", "Personal Care Items", "Childcare Support", "Technology Access", "Clothing & Weather Essentials",
    "Academic Support", "Community & Belonging", "School Supplies", "Cooking Supplies", "Cleaning Supplies",
    "Nutrition Education", "Financial Literacy", "Legal Support", "Crisis Intervention", "Laundry Access",
    "Career Resources", "Substance Abuse Support", "Financial Counseling", "Emergency Housing", 
    "Immigration & International Student Support", "Communication Services", "Domestic Violence Resources"
]


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

data = []

def clean_text(text):
    """Removes excessive spaces, newlines, and special characters from text."""
    return re.sub(r'\s+', ' ', text).strip()

#updated extraction function
def extract_relevant_text(url):
    """Extracts relevant content and retrieves keyword occurrences with sentence context."""
    
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    text = clean_text(soup.get_text().lower())
    extracted_info = {"URL": url, "Text": text}

    # Process text with spaCy
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]  # Tokenize into sentences

    #loop through the keyword list
    for keyword in keywords:
        keyword_lower = keyword.lower()
        occurrences = []
        
        # Find occurences
        for i, sentence in enumerate(sentences):
            if keyword_lower in sentence.lower():  # If the keyword is found in the sentence
                before = sentences[i - 1] if i > 0 else "N/A"  # Previous sentence
                after = sentences[i + 1] if i < len(sentences) - 1 else "N/A"  # Next sentence
                occurrence_text = f"Occurrence {len(occurrences) + 1}: Before: {before} | Within: {sentence} | After: {after}"
                occurrences.append(occurrence_text)

        # Placing the occurences into the same column separated by ||
        extracted_info[keyword] = " || ".join(occurrences) if occurrences else "No"

    return extracted_info


# Old extraction function
# def extract_relevant_text(url):
#     """Extracts relevant content based on predefined keywords."""
#     driver.get(url)
#     time.sleep(3)
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     text = clean_text(soup.get_text().lower())
#     extracted_info = {"URL": url, "Text": text}
    
#     for keyword in keywords:
#         extracted_info[keyword] = "Yes" if keyword.lower() in text else "No"
    
#     return extracted_info

def extract_contact_info(text):
    """Extracts email and phone numbers from the scraped text."""
    emails = re.findall(r'[\w\.-]+@[\w\.-]+', text)
    phones = re.findall(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text)
    return {"Emails": ", ".join(set(emails)), "Phone Numbers": ", ".join(set(phones))}

# old categorize function
# def categorize_services(text):
#     """Uses NLP to categorize extracted text into relevant service categories."""
#     doc = nlp(text)
#     categories = {key: "No" for key in keywords}
    
#     for sent in doc.sents:
#         for key in keywords:
#             if key.lower() in sent.text.lower():
#                 categories[key] = "Yes"
#     return categories

print("Starting enhanced web scraping...")
for url in university_urls:
    try:
        #print(f"Scraping: {url}")
        extracted_data = extract_relevant_text(url)
        contact_info = extract_contact_info(extracted_data["Text"])
#         categorized_data = categorize_services(extracted_data["Text"])
        
        # Merge all extracted data and got rid of the categorized data
        final_data = {**extracted_data, **contact_info}  #, **categorized_data
        data.append(final_data)
    except Exception as e:
        print(f"Error scraping {url}: {e}")

driver.quit()

# Convert to DataFrame and save as CSV
df = pd.DataFrame(data)
df.drop(columns=["Text"], inplace=True) 
df["school_name"] = result["school_name"] # Remove raw text to keep CSV clean

#df


#Aggregate keyword occurrences into a single row per school
def merge_occurrences(series):
    """Merge occurrences from multiple rows into a single row, separated by ' || '."""
    return " || ".join(series.dropna().unique()) if not series.isnull().all() else "No"

#Count total mentions per school
def count_mentions(series):
    """Count total keyword mentions across multiple rows for a school."""
    return series.str.count("Occurrence").sum()

#Perform groupby aggregation
agg_dict = {keyword: merge_occurrences for keyword in keywords}
agg_dict["Emails"] = merge_occurrences
agg_dict["Phone Numbers"] = merge_occurrences

df_grouped = df.groupby("school_name").agg(agg_dict).reset_index()

#Create a new column for total keyword mentions
df_grouped["Total Mentions"] = df[keywords].applymap(lambda x: x.count("Occurrence") if isinstance(x, str) else 0).groupby(df["school_name"]).sum().sum(axis=1).values

df_grouped


Starting enhanced web scraping...


,school_name,Food Security,Housing Stability,Financial Assistance,Healthcare Services,Mental Health Support,Transportation Access,Personal Care Items,Childcare Support,Technology Access,...,Career Resources,Substance Abuse Support,Financial Counseling,Emergency Housing,Immigration & International Student Support,Communication Services,Domestic Violence Resources,Emails,Phone Numbers,Total Mentions
0,PSU,No,No,No || Occurrence 1: Before: first-gen advocate...,No,No,No,No,No,No,...,No,No,No,No,No,No,No,"prcc@psu.edu || || more@studentlifepsujan, mo...",814-865-3776 || || 814-863-2020 || 814-865-65...,48
1,UC Davis,No || Occurrence 1: Before: housing aggie comp...,No,No || Occurrence 1: Before: students from los ...,No,No,No,No,No,No,...,No,No,No,No,No,No,No,|| webmaster@ecsi.net || fas-ecrteam@ou.ad3.u...,"530-752-1011 || (888) 549-3274, 530-752-1011 |...",4
2,UCLA,No,No,No || Occurrence 1: Before: participants meet ...,Occurrence 1: Before: 1 on 1 consult add/adhd ...,No || Occurrence 1: Before: the star program i...,No,No,No,No,...,No,No,No,Occurrence 1: Before: food housing personal cr...,No,No,No,|| basicneeds@cpo.ucla.edu || info@caps.ucla.edu,"(310) 825-4321 || (310) 825-4321, 310-825-0768...",156
3,UGA,No,No,No || Occurrence 1: Before: sgaclothingcloset@...,No,No,No,No || Occurrence 1: Before: students can help ...,No,No,...,No,No,No,No,No,No,No,"|| bulldogbasics@uga.edu, sgafreshexpress@uga...","|| (706) 542-8479, 706-542-8479 || (706) 542-...",6
4,UNC,Occurrence 1: Before: a sampling of those depa...,No,No || Occurrence 1: Before: what resources are...,No,No,No,No,No,No,...,No,No,No,No || Occurrence 1: Before: the goal is to pro...,No,No,No,"dos@unc.edu, centercvrc@unc.edu || dos@unc.edu...","919-966-4042, 919-962-9640 || 919-962-8100, 91...",24


In [ ]:
#df.to_csv("first_word_search.csv", index=False)
df_grouped.to_csv("condensed_word_search.csv", index=False)